### Importing required modules

In [28]:
# import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
import PyPDF2

In [2]:
# loading the env variables
load_dotenv()

True

In [29]:
# loading the google gemini pro LLM model

GOOGLE_API_KEY = os.getenv('API_KEY')
# genai.configure(api_key=GOOGLE_API_KEY)
# model = genai.GenerativeModel('gemini-pro',generation_config={'temperature':0.3})
model = ChatGoogleGenerativeAI(model="gemini-pro",
                               google_api_key=GOOGLE_API_KEY,
                               temperature=0.3)

### Creating Prompt

In [38]:
Template = '''\
You are an expert in the creation of multi choice questions. \
Your job is create a {number} number of multi choice questions in {difficulty} level. \
Make sure the questions are not repeated and follow the below format for creating questions: \n
    1. first multi choice question.\n
    Ans: \n\tA. "first choice here."\n\tB. "second choice here."\n\tC. "third choice here."\n\tD. "fourth choice here."\n
    Correct: "correct answer".\n
    2. second multi choice question.\n
    Ans: \n\tA. "first choice here."\n\tB. "second choice here."\n\tC. "third choice here."\n\tD. "fourth choice here."\n
    Correct: "correct answer".\n
Using following text to generate muti choice questions based on above instructions:
"Text" : {text}
'''

In [39]:
print(Template)

You are an expert in the creation of multi choice questions. Your job is create a {number} number of multi choice questions in {difficulty} level. Make sure the questions are not repeated and follow the below format for creating questions: 

    1. first multi choice question.

    Ans: 
	A. "first choice here."
	B. "second choice here."
	C. "third choice here."
	D. "fourth choice here."

    Correct: "correct answer".

    2. second multi choice question.

    Ans: 
	A. "first choice here."
	B. "second choice here."
	C. "third choice here."
	D. "fourth choice here."

    Correct: "correct answer".

Using following text to generate muti choice questions based on above instructions:
"Text" : {text}



In [40]:
mcq_prompt = PromptTemplate(
    input_variables=['number','difficulty','text'],
    template=Template
)

In [48]:
base_chain = LLMChain(llm=model,
                      prompt=mcq_prompt,
                      output_key="mcq",
                      verbose=False,
                      )

In [49]:
input_text = '''
What Is Machine Learning?
Machine learning (ML) is a discipline of artificial intelligence (AI) that provides machines with the ability to automatically learn from data and past experiences while identifying patterns to make predictions with minimal human intervention.

Machine learning methods enable computers to operate autonomously without explicit programming. ML applications are fed with new data, and they can independently learn, grow, develop, and adapt.

Machine learning derives insightful information from large volumes of data by leveraging algorithms to identify patterns and learn in an iterative process. ML algorithms use computation methods to learn directly from data instead of relying on any predetermined equation that may serve as a model.

The performance of ML algorithms adaptively improves with an increase in the number of available samples during the ‘learning’ processes. For example, deep learning is a sub-domain of machine learning that trains computers to imitate natural human traits like learning from examples. It offers better performance parameters than conventional ML algorithms.

While machine learning is not a new concept dating back to World War II when the Enigma Machine was used – the ability to apply complex mathematical calculations automatically to growing volumes and varieties of available data is a relatively recent development.

Today, with the rise of big data, IoT, and ubiquitous computing, machine learning has become essential for solving problems across numerous areas, such as

Computational finance (credit scoring, algorithmic trading)
Computer vision (facial recognition, motion tracking, object detection)
Computational biology (DNA sequencing, brain tumor detection, drug discovery)
Automotive, aerospace, and manufacturing (predictive maintenance)
Natural language processing (voice recognition)
How does machine learning work?
Machine learning algorithms are molded on a training dataset to create a model. As new input data is introduced to the trained ML algorithm, it uses the developed model to make a prediction. The prediction is checked for accuracy. Based on its accuracy, the ML algorithm is either deployed or trained repeatedly with an augmented training dataset until the desired accuracy is achieved.
Types of Machine Learning
Machine learning algorithms can be trained in many ways, with each method having its pros and cons. Based on these methods and ways of learning, machine learning is broadly categorized into four main types:

Types of Machine LearningTypes of Machine Learning
1. Supervised machine learning
This type of ML involves supervision, where machines are trained on labeled datasets and enabled to predict outputs based on the provided training. The labeled dataset specifies that some input and output parameters are already mapped. Hence, the machine is trained with the input and corresponding output. A device is made to predict the outcome using the test dataset in subsequent phases.

For example, consider an input dataset of parrot and crow images. Initially, the machine is trained to understand the pictures, including the parrot and crow’s color, eyes, shape, and size. Post-training, an input picture of a parrot is provided, and the machine is expected to identify the object and predict the output. The trained machine checks for the various features of the object, such as color, eyes, shape, etc., in the input picture, to make a final prediction. This is the process of object identification in supervised machine learning.

The primary objective of the supervised learning technique is to map the input variable (a) with the output variable (b). Supervised machine learning is further classified into two broad categories:

Classification: These refer to algorithms that address classification problems where the output variable is categorical; for example, yes or no, true or false, male or female, etc. Real-world applications of this category are evident in spam detection and email filtering.
Some known classification algorithms include the Random Forest Algorithm, Decision Tree Algorithm, Logistic Regression Algorithm, and Support Vector Machine Algorithm.

Regression: Regression algorithms handle regression problems where input and output variables have a linear relationship. These are known to predict continuous output variables. Examples include weather prediction, market trend analysis, etc.
Popular regression algorithms include the Simple Linear Regression Algorithm, Multivariate Regression Algorithm, Decision Tree Algorithm, and Lasso Regression.

2. Unsupervised machine learning
Unsupervised learning refers to a learning technique that’s devoid of supervision. Here, the machine is trained using an unlabeled dataset and is enabled to predict the output without any supervision. An unsupervised learning algorithm aims to group the unsorted dataset based on the input’s similarities, differences, and patterns.

For example, consider an input dataset of images of a fruit-filled container. Here, the images are not known to the machine learning model. When we input the dataset into the ML model, the task of the model is to identify the pattern of objects, such as color, shape, or differences seen in the input images and categorize them. Upon categorization, the machine then predicts the output as it gets tested with a test dataset.

Unsupervised machine learning is further classified into two types:

Clustering: The clustering technique refers to grouping objects into clusters based on parameters such as similarities or differences between objects. For example, grouping customers by the products they purchase.
Some known clustering algorithms include the K-Means Clustering Algorithm, Mean-Shift Algorithm, DBSCAN Algorithm, Principal Component Analysis, and Independent Component Analysis.

Association: Association learning refers to identifying typical relations between the variables of a large dataset. It determines the dependency of various data items and maps associated variables. Typical applications include web usage mining and market data analysis.
Popular algorithms obeying association rules include the Apriori Algorithm, Eclat Algorithm, and FP-Growth Algorithm.

3. Semi-supervised learning
Semi-supervised learning comprises characteristics of both supervised and unsupervised machine learning. It uses the combination of labeled and unlabeled datasets to train its algorithms. Using both types of datasets, semi-supervised learning overcomes the drawbacks of the options mentioned above.

Consider an example of a college student. A student learning a concept under a teacher’s supervision in college is termed supervised learning. In unsupervised learning, a student self-learns the same concept at home without a teacher’s guidance. Meanwhile, a student revising the concept after learning under the direction of a teacher in college is a semi-supervised form of learning. 

4. Reinforcement learning
Reinforcement learning is a feedback-based process. Here, the AI component automatically takes stock of its surroundings by the hit & trial method, takes action, learns from experiences, and improves performance. The component is rewarded for each good action and penalized for every wrong move. Thus, the reinforcement learning component aims to maximize the rewards by performing good actions.

Unlike supervised learning, reinforcement learning lacks labeled data, and the agents learn via experiences only. Consider video games. Here, the game specifies the environment, and each move of the reinforcement agent defines its state. The agent is entitled to receive feedback via punishment and rewards, thereby affecting the overall game score. The ultimate goal of the agent is to achieve a high score.

Reinforcement learning is applied across different fields such as game theory, information theory, and multi-agent systems. Reinforcement learning is further divided into two types of methods or algorithms:

Positive reinforcement learning: This refers to adding a reinforcing stimulus after a specific behavior of the agent, which makes it more likely that the behavior may occur again in the future, e.g., adding a reward after a behavior.
Negative reinforcement learning: Negative reinforcement learning refers to strengthening a specific behavior that avoids a negative outcome.
'''

In [50]:
response = base_chain.run(number = 5,
                        difficulty = 'Easy',
                        text = input_text)

In [51]:
print(response)

1. What is the main goal of machine learning?

Ans: 
	A. To provide machines with the ability to automatically learn from data and past experiences.
	B. To enable computers to operate autonomously without explicit programming.
	C. To derive insightful information from large volumes of data.
	D. To improve the performance of ML algorithms with an increase in the number of available samples.

Correct: A. To provide machines with the ability to automatically learn from data and past experiences.

2. Which type of machine learning involves supervision, where machines are trained on labeled datasets?

Ans: 
	A. Supervised machine learning
	B. Unsupervised machine learning
	C. Semi-supervised learning
	D. Reinforcement learning

Correct: A. Supervised machine learning

3. What is the objective of supervised machine learning?

Ans: 
	A. To map the input variable (a) with the output variable (b).
	B. To group the unsorted dataset based on the input’s similarities, differences, and patterns.
	C